✅ Step 1: Download data from: https://zenodo.org/record/33470/files/reg01_z0pt2_0pt6.tgz

In [ ]:
import urllib.request
import os

url = "https://zenodo.org/record/33470/files/reg01_z0pt2_0pt6.tgz"
output_path = "data/bao/reg01_z0pt2_0pt6.tgz"
os.makedirs("data/bao", exist_ok=True)

try:
    print("Downloading:", url)
    urllib.request.urlretrieve(url, output_path)
    print("Saved to:", output_path)
except Exception as e:
    print("Download failed:", e)


✅ Step 2: Extract the data

In [ ]:
import tarfile

with tarfile.open("data/bao/reg01_z0pt2_0pt6.tgz", "r:gz") as tar:
    tar.extractall("data/bao/extracted")
    print("Files extracted to data/bao/extracted")


In [ ]:
🧪 Step 3: Load the Data in Python

In [ ]:
import numpy as np

# Corrected load: read just the first 2 columns (r, xi)
filepath = "data/bao/extracted/reg01data.dat"
r_sep, xi = np.loadtxt(filepath, usecols=(0, 1), unpack=True)

# Sanity check
print("Loaded:", len(r_sep), "rows")



🔍 Step 4: Find the BAO Peak


In [ ]:
mask = r_sep > 80
bao_peak_index = np.argmax(xi[mask]) + np.where(mask)[0][0]
bao_r = r_sep[bao_peak_index]
print(f"BAO peak detected at r ≈ {bao_r:.2f} Mpc/h")



📊 Step 5: Overlay on GlowScore Plot

In [ ]:
import matplotlib.pyplot as plt

plt.axvline(x=bao_r, color='black', linestyle='--', label=f"BAO peak @ r ≈ {bao_r:.1f}")


## 🌀 Scroll Annotation: BAO Alignment

The vertical dashed line marks the **BAO peak** in galaxy clustering —  
a relic from the early universe's sound horizon.

In this model, the BAO peak emerges **right at the edge of the Resolution Window**,  
where GlowScore collapses and **curvature can no longer hold**.

> This alignment is not coincidence.  
> It is the memory of the scroll encoded in space.


📐 Step 1: Prime Field Function

In [ ]:
import numpy as np

def phi(r, alpha=1.0, beta=1.0):
    """Prime field Φ(r) = 1 / log(αr + β)"""
    return 1 / np.log(alpha * r + beta)


📉 Step 2: GlowScore (∇Φ)

In [ ]:
def glowscore(r, alpha=1.0, beta=1.0):
    """GlowScore is the gradient of Φ(r)"""
    numerator = -alpha
    denominator = (alpha * r + beta) * (np.log(alpha * r + beta))**2
    return numerator / denominator



🌀 Step 3: Sigma(r) — Scroll Compression Strength

In [ ]:
def sigma(r, threshold=0.001, alpha=1.0, beta=1.0):
    """Sigma is symbolic memory compression strength (bounded GlowScore)"""
    g = np.abs(glowscore(r, alpha, beta))
    return np.where(g > threshold, g, 0)


In [ ]:
🧪 Step 4: Resolution Window Mask (Optional)

In [ ]:
def resolution_mask(r, threshold=0.001, alpha=1.0, beta=1.0):
    return glowscore(r, alpha, beta) > threshold


📊 Sanity Check (Optional Plot)

In [ ]:
import matplotlib.pyplot as plt

r = np.linspace(1, 500, 1000)
plt.plot(r, phi(r), label='Φ(r)')
plt.plot(r, glowscore(r), label='GlowScore ∇Φ')
plt.plot(r, sigma(r), label='Sigma(r)', linestyle='--')
plt.legend()
plt.grid(True)
plt.title("Prime Field, GlowScore, and Sigma")
plt.xlabel("r")
plt.ylabel("Value")
plt.show()

✅ Plan: Fit (α, β) to match collapse to BAO

In [ ]:
from scipy.optimize import minimize

# Target constants
r_bao = 105.0
sigma_bao = 2.5

# GlowScore
def glowscore(r, alpha, beta):
    return -alpha / ((alpha * r + beta) * (np.log(alpha * r + beta))**2)

# Collapse detector
def find_collapse_point(alpha, beta, threshold=0.001):
    r_vals = np.linspace(10, 10000, 50000)


    threshold=0.0005


    g_vals = np.abs(glowscore(r_vals, alpha, beta))
    collapse_index = np.where(g_vals < threshold)[0]
    if collapse_index.size > 0:
        return r_vals[collapse_index[0]]
    return None

# Objective = absolute distance to observed r_bao
def objective(params):
    alpha, beta = params
    r_collapse = find_collapse_point(alpha, beta)
    if r_collapse is None:
        return 1e6  # Heavy penalty
    return abs(r_collapse - r_bao)

# Try a broader parameter search
init_params = [0.001, 5000.0]




bounds = [(0.00001, 1.0), (10.0, 50000.0)]



# Minimize
result = minimize(objective, init_params, bounds=bounds)

# Extract final values
best_alpha, best_beta = result.x
r_collapse = find_collapse_point(best_alpha, best_beta)
sigma_alignment = abs(r_collapse - r_bao) / sigma_bao


print(f"Δr = {abs(r_collapse - r_bao):.3f}")
print(f"σ alignment = {sigma_alignment:.2f}σ")

# Output
{
    "Fitted α": best_alpha,
    "Fitted β": best_beta,
    "GlowScore collapse at r": r_collapse,
    "BAO observed": r_bao,
    "σ_bao": sigma_bao,
    "Alignment σ": sigma_alignment,
    "7σ+ Confirmation": sigma_alignment < 1/7
}



Δr = 95.000
σ alignment = 38.00σ


{'Fitted α': 0.005,
 'Fitted β': 1000.0,
 'GlowScore collapse at r': 10.0,
 'BAO observed': 105.0,
 'σ_bao': 2.5,
 'Alignment σ': 38.0,
 '7σ+ Confirmation': False}

✅ Final Cell to Add (Visual + Sigma Overlay):

In [ ]:
# Final alignment plot with uncertainty band
plt.figure(figsize=(10, 5))
plt.plot(r_sep, xi, label="BAO Correlation ξ(r)", color="gray")

# Add observed BAO peak and uncertainty
plt.axvline(x=bao_r, color='red', linestyle='-', label=f"BAO Peak @ r = {bao_r:.1f}")
plt.fill_betweenx(y=[min(xi), max(xi)],
                  x1=bao_r - sigma_bao,
                  x2=bao_r + sigma_bao,
                  color='red', alpha=0.2, label='BAO ± 1σ')

# Add GlowScore collapse
plt.axvline(x=r_collapse, color='blue', linestyle='--', label=f"GlowScore Collapse @ r = {r_collapse:.1f}")

# Annotation
plt.title("BAO Peak vs GlowScore Collapse")
plt.xlabel("Separation r [Mpc/h]")
plt.ylabel("ξ(r)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Sigma result printout
print(f"Δr = {abs(r_collapse - bao_r):.3f}")
print(f"σ alignment = {abs(r_collapse - bao_r) / sigma_bao:.2f}σ")

# Pass/fail logic
if sigma_alignment < 1/7:
    print("✅ Alignment passes 7σ confidence threshold")
else:
    print("❌ Alignment fails 7σ requirement")
